In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import numpy as np
from scipy.spatial import distance

from tqdm.auto import tqdm
import pickle
import gc
import json
import h5py
import pandas as pd

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

# == recnn ==
import sys
sys.path.append("../../")
import recnn


cuda = torch.device('cuda')
frame_size = 10
# https://drive.google.com/open?id=1xjVI4uVQGsQ7tjOJ3594ZXmAEC_6yX0e
meta = json.load(open('../../data/parsed/omdb.json'))
tqdm.pandas()

In [3]:
env = recnn.env.FrameEnv('../../data/embeddings/ml20_pca128.pkl',
                         '../../data/ml-20m/ratings.csv', 10, 1)

In [4]:
ddpg = recnn.models.Actor(1290, 128, 256).to(cuda)
td3 = recnn.models.Actor(1290, 128, 256).to(cuda)
ddpg.load_state_dict(torch.load('../../models/ddpg_policy.pt'))
td3.load_state_dict(torch.load('../../models/td3_policy.pt'))

<All keys matched successfully>

In [6]:
test_batch = next(iter(env.test_dataloader))
state = test_batch[0].to(cuda)

In [10]:
def rank(gen_action, metric):
    scores = []
    for i in env.movie_embeddings_key_dict.keys():
        scores.append([i, metric(env.movie_embeddings_key_dict[i], gen_action)])
    scores = list(sorted(scores, key = lambda x: x[1]))
    scores = scores[:10]
    ids = [i[0] for i in scores]
    for i in range(10):
        scores[i].extend([meta[str(scores[i][0])]['omdb'][key]  for key in ['Title',
                                'Genre', 'Language', 'Released', 'imdbRating']])
        # scores[i][3] = ' '.join([genres_dict[i] for i in scores[i][3]])

    indexes = ['id', 'score', 'Title', 'Genre', 'Language', 'Released', 'imdbRating']
    table_dict = dict([(key,[i[idx] for i in scores]) for idx, key in enumerate(indexes)])
    table = pd.DataFrame(table_dict)
    return table

# DDPG

In [11]:
ddpg_action = ddpg(state)
# pick random action
ddpg_action = ddpg_action[np.random.randint(0, state.size(0), 1)[0]].detach().cpu().numpy()

In [12]:
from pandas.plotting import table
import subprocess
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(ddpg_action, distance.euclidean)

,id,score,Title,Genre,Language,Released,imdbRating
0,3965,48.736301,The Strange Love of Martha Ivers,"Drama, Film-Noir, Romance",English,13 Sep 1946,7.5
1,1226,49.293373,The Quiet Man,"Comedy, Drama, Romance","English, Irish",14 Sep 1952,7.8
2,869,49.358780,Kansas City,"Crime, Drama, Music, Thriller",English,16 Aug 1996,6.4
3,86418,49.368202,City Streets,"Crime, Drama, Film-Noir, Romance",English,18 Apr 1931,7.2
4,121797,49.431633,The Old Maid,Drama,English,02 Sep 1939,7.7
5,32361,49.451637,Come and Get It,"Drama, Romance",English,23 Dec 1936,7.0
6,77615,49.483135,The Dark Tower,"Drama, Thriller",English,18 Oct 1943,6.1
7,112395,49.541718,Belle Starr,Western,English,12 Sep 1941,5.8
8,125417,49.575157,Stars and Stripes Forever,"Biography, Comedy, Music",English,16 Feb 1953,7.3
9,8828,49.665157,Dead Ringer,"Crime, Drama, Thriller",English,19 Feb 1964,7.4


In [13]:
rank(ddpg_action, distance.cosine)

/home/dev/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


,id,score,Title,Genre,Language,Released,imdbRating
0,3965,0.664612,The Strange Love of Martha Ivers,"Drama, Film-Noir, Romance",English,13 Sep 1946,7.5
1,109788,0.680811,Bad Country,"Action, Crime, Drama, Mystery, Thriller",English,29 Apr 2014,5.8
2,77615,0.684950,The Dark Tower,"Drama, Thriller",English,18 Oct 1943,6.1
3,125417,0.689441,Stars and Stripes Forever,"Biography, Comedy, Music",English,16 Feb 1953,7.3
4,32361,0.689847,Come and Get It,"Drama, Romance",English,23 Dec 1936,7.0
5,121797,0.691234,The Old Maid,Drama,English,02 Sep 1939,7.7
6,8828,0.692851,Dead Ringer,"Crime, Drama, Thriller",English,19 Feb 1964,7.4
7,1226,0.694081,The Quiet Man,"Comedy, Drama, Romance","English, Irish",14 Sep 1952,7.8
8,6826,0.696981,The Shakiest Gun in the West,"Comedy, Western",English,10 Jul 1968,6.4
9,105320,0.697319,Apache Country,"Action, Music, Western",English,30 May 1952,6.4


In [14]:
rank(ddpg_action, distance.correlation) # looks similar to cosine

,id,score,Title,Genre,Language,Released,imdbRating
0,77615,0.666476,The Dark Tower,"Drama, Thriller",English,18 Oct 1943,6.1
1,3965,0.667414,The Strange Love of Martha Ivers,"Drama, Film-Noir, Romance",English,13 Sep 1946,7.5
2,32361,0.682182,Come and Get It,"Drama, Romance",English,23 Dec 1936,7.0
3,109788,0.685629,Bad Country,"Action, Crime, Drama, Mystery, Thriller",English,29 Apr 2014,5.8
4,125417,0.690667,Stars and Stripes Forever,"Biography, Comedy, Music",English,16 Feb 1953,7.3
5,6826,0.692258,The Shakiest Gun in the West,"Comedy, Western",English,10 Jul 1968,6.4
6,105320,0.693204,Apache Country,"Action, Music, Western",English,30 May 1952,6.4
7,121797,0.694728,The Old Maid,Drama,English,02 Sep 1939,7.7
8,1226,0.697950,The Quiet Man,"Comedy, Drama, Romance","English, Irish",14 Sep 1952,7.8
9,8460,0.701102,State Fair,"Comedy, Drama, Musical, Romance",English,30 Aug 1945,7.1


In [15]:
rank(ddpg_action, distance.canberra)

,id,score,Title,Genre,Language,Released,imdbRating
0,80164,83.660743,Burrowing,Drama,Swedish,28 Aug 2009,5.9
1,94301,84.213645,The Marc Pease Experience,"Comedy, Drama, Music",English,21 May 2010,4.1
2,86334,84.436754,In Old Oklahoma,"Romance, Western",English,06 Dec 1943,6.6
3,130614,85.223532,Goodbye Uncle Tom,Drama,"Italian, English",30 Sep 1971,6.8
4,80763,85.322895,Saint John of Las Vegas,"Comedy, Drama",English,10 Jun 2009,5.7
5,90462,85.373588,Alaska: Silence & Solitude,Documentary,English,N/A,7.4
6,115287,85.564112,Stage Fright,"Comedy, Horror, Musical",English,03 Apr 2014,5.2
7,129413,85.740321,In the Name of the Law,"Crime, Drama",Italian,19 Feb 1949,7.5
8,108656,85.801578,Report to the Commissioner,"Crime, Drama",English,19 Sep 1975,6.4
9,96919,85.915250,Attack of the 50 Foot Cheerleader,Comedy,English,N/A,4.5


In [16]:
rank(ddpg_action, distance.minkowski)

,id,score,Title,Genre,Language,Released,imdbRating
0,3965,48.736301,The Strange Love of Martha Ivers,"Drama, Film-Noir, Romance",English,13 Sep 1946,7.5
1,1226,49.293373,The Quiet Man,"Comedy, Drama, Romance","English, Irish",14 Sep 1952,7.8
2,869,49.358780,Kansas City,"Crime, Drama, Music, Thriller",English,16 Aug 1996,6.4
3,86418,49.368202,City Streets,"Crime, Drama, Film-Noir, Romance",English,18 Apr 1931,7.2
4,121797,49.431633,The Old Maid,Drama,English,02 Sep 1939,7.7
5,32361,49.451637,Come and Get It,"Drama, Romance",English,23 Dec 1936,7.0
6,77615,49.483135,The Dark Tower,"Drama, Thriller",English,18 Oct 1943,6.1
7,112395,49.541718,Belle Starr,Western,English,12 Sep 1941,5.8
8,125417,49.575157,Stars and Stripes Forever,"Biography, Comedy, Music",English,16 Feb 1953,7.3
9,8828,49.665157,Dead Ringer,"Crime, Drama, Thriller",English,19 Feb 1964,7.4


In [17]:
rank(ddpg_action, distance.chebyshev)

,id,score,Title,Genre,Language,Released,imdbRating
0,4753,10.998600,Vamp,"Comedy, Fantasy, Horror",English,18 Jul 1986,5.9
1,3835,11.028734,The Crush,"Drama, Thriller",English,02 Apr 1993,5.8
2,50585,11.046098,Juarez,"Biography, Drama, History, Romance","English, Spanish",10 Jun 1939,7.3
3,4579,11.069529,Cookie,"Comedy, Crime",English,23 Aug 1989,5.3
4,39307,11.087187,Dreamer: Inspired by a True Story,"Drama, Family, Sport",English,21 Oct 2005,6.9
5,73876,11.179543,Undisputed 2: Last Man Standing,"Action, Crime, Drama, Sport","English, Russian",28 Feb 2007,7.1
6,98755,11.247617,1911,"Action, Adventure, Drama, History, War","Mandarin, English, French",23 Sep 2011,5.8
7,32515,11.271762,Walker,"Biography, Drama, History, Western","English, American Sign Language, Spanish",04 Dec 1987,6.6
8,120783,11.348541,Son of a Gun,"Action, Crime, Drama, Thriller","English, Serbo-Croatian",11 Dec 2014,6.5
9,91241,11.361153,Singham,"Action, Drama","Hindi, Marathi, Gujarati, Tulu, English",22 Jul 2011,6.8


In [18]:
rank(ddpg_action, distance.braycurtis)

,id,score,Title,Genre,Language,Released,imdbRating
0,109788,0.771752,Bad Country,"Action, Crime, Drama, Mystery, Thriller",English,29 Apr 2014,5.8
1,97236,0.779573,Shadowing the Third Man,Documentary,English,11 Oct 2004,7.0
2,66639,0.781668,Street Fighter: The Legend of Chun-Li,"Action, Crime, Thriller","English, Hindi, Japanese, Thai",27 Feb 2009,3.7
3,7570,0.790882,Octopussy,"Action, Adventure, Thriller","English, Russian, German, Spanish, Hindi",10 Jun 1983,6.6
4,130614,0.794294,Goodbye Uncle Tom,Drama,"Italian, English",30 Sep 1971,6.8
5,27163,0.795214,Kuch Kuch Hota Hai,"Comedy, Drama, Musical, Romance","Hindi, Urdu, English, Italian",16 Oct 1998,7.7
6,84680,0.795548,My Foolish Heart,"Drama, Romance",English,21 Jan 1950,7.1
7,111868,0.796754,Best Foot Forward,"Musical, Comedy",English,08 Oct 1943,6.6
8,86334,0.798950,In Old Oklahoma,"Romance, Western",English,06 Dec 1943,6.6
9,125417,0.799029,Stars and Stripes Forever,"Biography, Comedy, Music",English,16 Feb 1953,7.3


In [19]:
rank(ddpg_action, distance.cityblock)

,id,score,Title,Genre,Language,Released,imdbRating
0,66639,438.094360,Street Fighter: The Legend of Chun-Li,"Action, Crime, Thriller","English, Hindi, Japanese, Thai",27 Feb 2009,3.7
1,125417,439.224915,Stars and Stripes Forever,"Biography, Comedy, Music",English,16 Feb 1953,7.3
2,109788,439.569580,Bad Country,"Action, Crime, Drama, Mystery, Thriller",English,29 Apr 2014,5.8
3,3452,440.854492,Romeo Must Die,"Action, Crime, Thriller","English, Chinese",22 Mar 2000,6.1
4,3873,441.541748,Cat Ballou,"Comedy, Romance, Western",English,24 Jun 1965,6.9
5,7570,441.776123,Octopussy,"Action, Adventure, Thriller","English, Russian, German, Spanish, Hindi",10 Jun 1983,6.6
6,48142,442.114807,The Black Dahlia,"Crime, Drama, Mystery, Thriller","English, German",15 Sep 2006,5.6
7,99659,442.294495,The Sea of Grass,"Drama, Western",English,25 Apr 1947,6.4
8,3965,442.314758,The Strange Love of Martha Ivers,"Drama, Film-Noir, Romance",English,13 Sep 1946,7.5
9,869,443.482605,Kansas City,"Crime, Drama, Music, Thriller",English,16 Aug 1996,6.4


# TD3

In [30]:
td3_action = td3(state)
# pick random action
td3_action = td3_action[np.random.randint(0, state.size(0), 1)[0]].detach().cpu().numpy()

In [31]:
from pandas.plotting import table 
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(td3_action, distance.euclidean)

KeyError: '0'

In [32]:
rank(td3_action, distance.cosine)

,id,score,Title,Genre,Language,Released,imdbRating
0,100613,0.671745,Saving Lincoln,Biography,English,13 Feb 2013,5.3
1,7014,0.689951,Nowhere to Run,"Action, Drama, Romance, Thriller",English,15 Jan 1993,5.7
2,127206,0.703335,People Places Things,"Comedy, Romance",English,14 Aug 2015,6.9
3,89052,0.706398,The Cool School,Documentary,English,07 Mar 2008,7.1
4,5836,0.717264,Houseboat,"Comedy, Drama, Family, Romance","English, Italian",11 Jan 1959,6.7
5,113872,0.717742,Captive Heart: The James Mink Story,Drama,English,14 Apr 1996,7.1
6,43560,0.722370,Nanny McPhee,"Comedy, Family, Fantasy",English,27 Jan 2006,6.5
7,352,0.722616,Crooklyn,"Comedy, Drama",English,13 May 1994,6.9
8,92200,0.730484,The Sound and the Fury,Drama,English,06 Apr 1959,6.4
9,2701,0.730698,Wild Wild West,"Action, Comedy, Sci-Fi, Western",English,30 Jun 1999,4.9


In [33]:
rank(td3_action, distance.correlation) # looks similar to cosine

,id,score,Title,Genre,Language,Released,imdbRating
0,100613,0.673297,Saving Lincoln,Biography,English,13 Feb 2013,5.3
1,7014,0.692936,Nowhere to Run,"Action, Drama, Romance, Thriller",English,15 Jan 1993,5.7
2,127206,0.699368,People Places Things,"Comedy, Romance",English,14 Aug 2015,6.9
3,89052,0.706504,The Cool School,Documentary,English,07 Mar 2008,7.1
4,113872,0.716356,Captive Heart: The James Mink Story,Drama,English,14 Apr 1996,7.1
5,5836,0.718330,Houseboat,"Comedy, Drama, Family, Romance","English, Italian",11 Jan 1959,6.7
6,352,0.719955,Crooklyn,"Comedy, Drama",English,13 May 1994,6.9
7,43560,0.725333,Nanny McPhee,"Comedy, Family, Fantasy",English,27 Jan 2006,6.5
8,2701,0.729046,Wild Wild West,"Action, Comedy, Sci-Fi, Western",English,30 Jun 1999,4.9
9,41810,0.731867,Cinderella Liberty,"Drama, Romance",English,07 Apr 1974,6.8


In [34]:
rank(td3_action, distance.canberra)

,id,score,Title,Genre,Language,Released,imdbRating
0,110858,76.549930,Firstborn,"Drama, Thriller",English,26 Oct 1984,6.2
1,7370,77.562002,A Foreign Affair,"Comedy, Drama, Romance","English, Russian",21 Jan 2003,6.0
2,3397,77.956441,The Great Muppet Caper,"Adventure, Comedy, Crime, Family, Musical, Mys...",English,26 Jun 1981,7.2
3,5756,78.176850,Heartbeeps,"Sci-Fi, Comedy",English,18 Dec 1981,4.2
4,38992,78.888309,Two for the Money,"Drama, Sport, Thriller",English,07 Oct 2005,6.3
5,105911,78.938903,Heart of a Lion,"Drama, Romance",Finnish,18 Oct 2013,7.0
6,7814,78.985080,Waterloo,"Action, Biography, Drama, History, War",English,29 Oct 1970,7.3
7,47619,79.077736,State of the Union,"Comedy, Drama",English,30 Apr 1948,7.3
8,98083,79.279428,Jackass 3.5,"Action, Comedy",English,01 Apr 2011,6.8
9,95441,79.400466,Ted,"Comedy, Fantasy","English, Japanese",29 Jun 2012,6.9


In [35]:
rank(td3_action, distance.minkowski)

KeyError: '0'

In [ ]:
rank(td3_action, distance.chebyshev)

In [36]:
rank(td3_action, distance.braycurtis)

,id,score,Title,Genre,Language,Released,imdbRating
0,2701,0.705507,Wild Wild West,"Action, Comedy, Sci-Fi, Western",English,30 Jun 1999,4.9
1,77919,0.715048,The Chalk Garden,"Drama, Mystery",English,21 May 1964,7.5
2,110858,0.719948,Firstborn,"Drama, Thriller",English,26 Oct 1984,6.2
3,500,0.741191,Mrs. Doubtfire,"Comedy, Drama, Family",English,24 Nov 1993,7.0
4,100613,0.741319,Saving Lincoln,Biography,English,13 Feb 2013,5.3
5,103836,0.742467,Black & White & Sex,Drama,English,01 Mar 2012,5.7
6,2124,0.742863,The Addams Family,"Comedy, Fantasy","English, Italian, French",22 Nov 1991,6.9
7,5760,0.743945,Lady Chatterley's Lover,"Drama, Romance",English,07 May 1982,5.1
8,98083,0.744212,Jackass 3.5,"Action, Comedy",English,01 Apr 2011,6.8
9,56869,0.744353,Drained,"Comedy, Drama",Portuguese,23 Mar 2007,7.4


In [37]:
rank(td3_action, distance.cityblock)

KeyError: '0'

# BCQ

In [40]:
perturbator = recnn.models.bcqPerturbator(1290, 128, 256).to(cuda)
generator = recnn.models.bcqGenerator(1290, 128, 512).to(cuda)

perturbator.load_state_dict(torch.load('../../models/bcq_perturbator.pt'))
generator.load_state_dict(torch.load('../../models/bcq_generator.pt'))

FileNotFoundError: [Errno 2] No such file or directory: '../../models/bcq_policy.pt'

In [37]:
sampled_actions = generator.decode(state)
perturbed_actions= perturbator(state, sampled_actions)

In [38]:
bcq_action = perturbed_actions[np.random.randint(0,
                               sampled_actions.size(0), 1)[0]].detach().cpu().numpy()

In [39]:
rank(bcq_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,6745,9.515295,Shrimp on the Barbie,Comedy Romance,en,1990,5.9
1,1629,9.713503,The Matchmaker,Comedy Romance,en,1997,6.3
2,31047,9.857168,Jinxed!,Comedy,en,1982,4.3
3,5513,9.858193,Martin Lawrence Live: Runteldat,Comedy,en,2002,5.0
4,87683,9.869917,An Unexpected Love,Drama TV Movie,en,2003,5.9
5,47423,9.888137,Half Nelson,Drama,en,2006,6.7
6,78574,9.922365,Winter's Bone,Drama,en,2010,6.9
7,87270,9.933637,The Other Woman,Drama,en,2010,6.2
8,106897,9.955842,Stuck Between Stations,Drama Romance,en,2011,6.2
9,88823,9.963136,High Time,Comedy,en,1960,5.5


In [40]:
rank(bcq_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,106696,0.673634,Frozen,Animation Adventure Family,en,2013,7.3
1,42794,0.691185,Buried Alive,TV Movie Horror Thriller,en,1990,6.4
2,6615,0.691691,Freddy vs. Jason,Horror,en,2003,5.8
3,6745,0.693560,Shrimp on the Barbie,Comedy Romance,en,1990,5.9
4,87222,0.697007,Kung Fu Panda 2,Animation Family,en,2011,6.7
5,112776,0.698551,Bloodline,Horror,it,2011,1.0
6,78062,0.705234,Puppet Master vs Demonic Toys,Horror,en,2004,3.1
7,113674,0.709200,Proxy,Drama Horror Thriller,en,2013,5.9
8,40339,0.710282,Chicken Little,Animation Family Comedy,en,2005,5.8
9,95105,0.711013,Madagascar 3: Europe's Most Wanted,Animation Family Comedy,en,2012,6.5


In [41]:
rank(bcq_action, distance.correlation)

,id,score,title,genres,language,year,rating
0,106696,0.673739,Frozen,Animation Adventure Family,en,2013,7.3
1,42794,0.690291,Buried Alive,TV Movie Horror Thriller,en,1990,6.4
2,6615,0.691004,Freddy vs. Jason,Horror,en,2003,5.8
3,6745,0.693545,Shrimp on the Barbie,Comedy Romance,en,1990,5.9
4,87222,0.696411,Kung Fu Panda 2,Animation Family,en,2011,6.7
5,112776,0.698540,Bloodline,Horror,it,2011,1.0
6,78062,0.705381,Puppet Master vs Demonic Toys,Horror,en,2004,3.1
7,40339,0.707243,Chicken Little,Animation Family Comedy,en,2005,5.8
8,113674,0.708868,Proxy,Drama Horror Thriller,en,2013,5.9
9,95105,0.709236,Madagascar 3: Europe's Most Wanted,Animation Family Comedy,en,2012,6.5


In [42]:
rank(bcq_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,129456,78.408064,The Legend of Bloody Jack,Horror,en,2007,4.4
1,833,78.562517,High School High,Comedy,en,1996,5.4
2,101662,78.929579,Three Way,Crime Drama Thriller,en,2004,4.3
3,97462,79.319606,Hoodwinked Too! Hood VS. Evil,Comedy Animation Family,en,2011,5.0
4,85790,79.422699,Meek's Cutoff,Drama Western,en,2010,6.5
5,45576,79.646512,Sinbad of the Seven Seas,Adventure Family,en,1989,3.7
6,73488,79.841134,Blood: The Last Vampire,Action Adventure Horror Sci-Fi Thriller,en,2009,5.1
7,7051,79.941488,What's New Pussycat?,Comedy Romance,en,1965,6.3
8,5621,79.946517,The Tuxedo,Thriller Action Comedy Sci-Fi,en,2002,5.5
9,6178,80.201325,A Patch of Blue,Drama Romance,en,1965,8.0


In [43]:
rank(bcq_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,6745,9.515295,Shrimp on the Barbie,Comedy Romance,en,1990,5.9
1,1629,9.713503,The Matchmaker,Comedy Romance,en,1997,6.3
2,31047,9.857168,Jinxed!,Comedy,en,1982,4.3
3,5513,9.858193,Martin Lawrence Live: Runteldat,Comedy,en,2002,5.0
4,87683,9.869917,An Unexpected Love,Drama TV Movie,en,2003,5.9
5,47423,9.888137,Half Nelson,Drama,en,2006,6.7
6,78574,9.922365,Winter's Bone,Drama,en,2010,6.9
7,87270,9.933637,The Other Woman,Drama,en,2010,6.2
8,106897,9.955842,Stuck Between Stations,Drama Romance,en,2011,6.2
9,88823,9.963136,High Time,Comedy,en,1960,5.5


In [44]:
rank(bcq_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,78039,1.942187,Blue Valentine,Drama Romance,en,2010,6.9
1,106897,2.047636,Stuck Between Stations,Drama Romance,en,2011,6.2
2,6295,2.053344,Chasing Papi,Comedy Romance,en,2003,6.0
3,94192,2.059334,Devil's Diary,Drama Horror,en,2007,4.5
4,129011,2.066347,Eddie Izzard: Glorious,Comedy,en,1997,6.8
5,6788,2.088190,Angie,Comedy Drama Romance,en,1994,5.4
6,79226,2.093588,The Good Guy,Comedy Romance,en,2009,5.6
7,3916,2.101965,Remember the Titans,Drama,en,2000,7.6
8,2069,2.109872,The Trip to Bountiful,Drama,en,1985,7.4
9,50594,2.113705,Daisies,Comedy,cs,1966,7.7


In [45]:
rank(bcq_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,6745,0.712504,Shrimp on the Barbie,Comedy Romance,en,1990,5.9
1,97462,0.721611,Hoodwinked Too! Hood VS. Evil,Comedy Animation Family,en,2011,5.0
2,101662,0.727641,Three Way,Crime Drama Thriller,en,2004,4.3
3,40339,0.730188,Chicken Little,Animation Family Comedy,en,2005,5.8
4,63436,0.730517,Saw V,Horror Thriller Crime,en,2008,6.1
5,100577,0.733169,The Angel Levine,Drama,en,1970,0.0
6,90576,0.733863,What's Your Number?,Comedy Romance,en,2011,6.4
7,54144,0.735894,The Last Time,Drama Romance,en,2006,5.5
8,87270,0.739809,The Other Woman,Drama,en,2010,6.2
9,3640,0.741412,A King in New York,Comedy Drama,en,1957,7.0


In [46]:
rank(bcq_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,6745,83.819412,Shrimp on the Barbie,Comedy Romance,en,1990,5.9
1,833,86.041580,High School High,Comedy,en,1996,5.4
2,1629,86.602219,The Matchmaker,Comedy Romance,en,1997,6.3
3,87270,88.043610,The Other Woman,Drama,en,2010,6.2
4,47423,88.132385,Half Nelson,Drama,en,2006,6.7
5,975,88.555115,Something to Sing About,Comedy,en,1937,5.4
6,84557,88.556129,From Prada to Nada,Comedy Drama Romance,en,2011,5.1
7,87683,88.685097,An Unexpected Love,Drama TV Movie,en,2003,5.9
8,33917,88.829498,The Member of the Wedding,Drama,en,1952,6.9
9,88823,88.973022,High Time,Comedy,en,1960,5.5
